### Измерение длин извилистых линий

Реализация обобщенного метода Ю.С. Фролова

### Окружности

$$ (x - a)^{2} + (y - b)^{2} = r^2 $$
$$ x^{2} + y^{2} - 2ax - 2by + (b^{2} + a^{2}) = r^{2} $$
$$ A = -2a; B = -2b; C = b^{2} + a^{2}; $$

### 0. Вспомогательные функции и билиотеки

In [3]:
import math
import geojson
import shapefile
from geopy import distance

DPI = 150
RAD_1 = 5
EARTH_RAD = 6372800

# Вычисление расстояния между двумя точками
def dist_between_points(p0, p1):
    return math.sqrt((p1[0] - p0[0])**2.0 + (p1[1] - p0[1])**2.0)

# Вычисление центра и радиуса окружности, вписанной в три точки
def get_circle(p1, p2, p3):
    temp = p2[0] ** 2.0 + p2[1] ** 2.0
    
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2.0
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2.0
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])
    
    if det == 0:
        print('get_circle: det is 0')
        return 0
    
    cx = (bc * (p2[1] - p3[1]) - cd * (p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det
    
    radius = math.sqrt((cx - p1[0]) ** 2.0 + (cy - p1[1]) ** 2.0)
    return ((cx, cy), radius)

# Вычисление коэффициентов прямой, проходящей через точки
def get_line(p0, p1):
    if p0[0] - p1[0] == 0:
        k = 0
    else:
        k = (p0[1] - p1[1]) / (p0[0] - p1[0])
    b = p1[1] - k*p1[0]
    #print("-y + %.2f*x + %.2f = 0" % (k, b))
    return (k, b) 

# Построение перпендикулярая к прямой проходящего через заданную точку
def get_perpendicular_line_by_point(line, p0):
    if line[0] == 0:
        raise Exception('get_perpendicular_line_by_point: line[0] is 0')
    
    k = - 1.0 / line[0]  
    
    b = p0[1] - k * p0[0]
    #print("-y + %.2f*x + %.2f = 0" % (k, b))
    return (k, b)

# Поиск точки пересечения двух прямых
def get_intersection_point(line0, line1):
    x = (line1[1] - line0[1]) / (line0[0] - line1[0])
    
    if line0[0] - line1[0] == 0:
        raise Exception('get_intersection_point: line0[0] - line1[0] is 0')
    
    y = line0[0] * x + line0[1]
    return (x, y)

# Расстояние от прямой до точки
def get_distance_from_line_to_point(p, line):
    p_line = get_perpendicular_line_by_point(line, p)
    intersection_point = get_intersection_point(p_line, line)
    return dist_between_points(p, intersection_point)

# Вычисление отклонения точки от прямой
def get_deviation(p0, line):
    return line[0] * p0[0] + line[1] - p0[1]

# Вычисление границ изгибов с самой удаленной точкой
def coordinates_bends(coordinates):
    bends_array = [coordinates[0]]
    remote_of_bends_points = []
    
    remote_max = 0.0
    remote_max_point = (0, 0)
    remote_max_point_prev = (0, 0)
    
    last_deviation = 0.0
    for coordinate_this, coordinate_next in  zip(coordinates[1:], coordinates[2:]):
        line = get_line(bends_array[-1], coordinate_next)
        new_deviation = get_deviation(coordinate_this, line)
        remote_dist = get_distance_from_line_to_point(coordinate_this, line)
        
        if remote_dist > remote_max:
            remote_max = remote_dist
            remote_max_point_prev = remote_max_point
            remote_max_point = coordinate_this
            
        if last_deviation == 0:
            last_deviation = new_deviation
            
        elif last_deviation < 0:
            if new_deviation > 0:
                bends_array.append(coordinate_this)
                remote_of_bends_points.append(remote_max_point_prev)
                remote_max, remote_max_point = 0.0, (0, 0)
        else:
            if new_deviation < 0:
                bends_array.append(coordinate_this)
                remote_of_bends_points.append(remote_max_point_prev)
                remote_max, remote_max_point = 0.0, (0, 0)
                
    # Отдельно обрабатываем конец
    bends_array.append(coordinates[-1])
    remote_of_bends_points.append(remote_max_point)
    
    return (bends_array, remote_of_bends_points)

def get_circle_section_length_between_points(p0, p1, circle):
    # Длина хорды, образуемая крайними точками триплета
    chord_section = dist_between_points(p0, p1)
    # Вычисление длин сторон треугольника образуемых между центром и крайними точками триплета
    triangle_left_side = dist_between_points(circle[0], p1)
    triangle_right_side = dist_between_points(circle[0], p0)
    # Вычисление косинуса угла, лежащего напротив основания образованного треугольника (хорды)
    cos_angle = (triangle_left_side**2.0 + triangle_right_side**2.0 - chord_section**2.0) / (2.0 * triangle_left_side * triangle_right_side)
    # Длина дуги как произведение радиуса на угол
    return circle[1] * math.acos(cos_angle)

def haversine(coord1, coord2, curvature = 1):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*EARTH_RAD*math.atan2(math.sqrt(a), math.sqrt(1 - a))*curvature

def wgs_dist(coord1, coord2):
    return distance.geodesic(coord1, coord2, ellipsoid='WGS-84').meters

### 1. Извлечение данных из geojson/shp формата

In [4]:
# Если используется geojson:
with open('./latest_test/segments.geojson') as f:
    parsed_geojson = geojson.load(f)
    
selected_feature = 0
features = parsed_geojson['features'][selected_feature]
print(f"Count of features: {len(parsed_geojson['features'])}")
print(f"Selected feature: {selected_feature}")
geometry = features['geometry']
coordinates = geometry['coordinates'][0]

coordinates = [(x[1], x[0]) for x in coordinates]

# Если используется shp
# shape = shapefile.Reader("sever/Линия_кривая_отрезками.shp")
# coordinates = shape.shapes()[0].points

coordinates_len = len(coordinates)
print(f"Count of points: {coordinates_len}")

Count of features: 1
Selected feature: 0
Count of points: 1327


### 2. Вписывание окружностей в триплеты точек

Использование фиксированного минимального числа точек, через которые возможно провести окружность.

In [5]:
if coordinates_len % 3 > 0:
    remains_coordinates = coordinates[-(coordinates_len % 3):]
else:
    remains_coordinates = []

triple_coordinates = list(zip(coordinates, coordinates[1:], coordinates[2:]))[::2]

result_arcs = 0.0
result_sections = 0.0
result_wgs = 0.0

for triple in triple_coordinates:
        
    first_section = haversine(triple[0], triple[1])
    second_section = haversine(triple[1], triple[2])
    result_sections += first_section + second_section
    result_wgs += wgs_dist(triple[0], triple[1])
    result_wgs += wgs_dist(triple[1], triple[2])
    
    circle = get_circle(triple[0], triple[1], triple[2])
    
    if circle == 0:
        result_arcs += dist_between_points(triple[0], triple[2])
    else:
        arc_len = get_circle_section_length_between_points(triple[0], triple[2], circle)
        curvature = arc_len / dist_between_points(triple[0], triple[2])
        result_arcs += haversine(triple[0], triple[2], curvature)
    
if len(remains_coordinates) >= 1:
    dist = haversine(triple_coordinates[-1][2], remains_coordinates[0])
    result_arcs += dist
    result_sections += dist
    result_wgs += wgs_dist(triple_coordinates[-1][2], remains_coordinates[0])
    if len(remains_coordinates) == 2:
        dist = haversine(remains_coordinates[0], remains_coordinates[1])
        result_arcs += dist
        result_sections += dist
        result_wgs += wgs_dist(remains_coordinates[0], remains_coordinates[1])

print(f'Результат сложением длин дуг:\n\t{result_arcs} м')
print(f'Результат вычисления длины через расстояния между точками:\n\t{result_sections} м')
print(f'Результат вычисления геодезической длины на WGS-84:\n\t{result_wgs} м')

get_circle: det is 0
Результат сложением длин дуг:
	666816.5583287559
Результат вычисления длины через расстояния между точками:
	661882.819429927
Результат вычисления геодезической длины на WGS-84:
	663696.2246929576
